In [40]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def system_model(R, k, sig, a_matrix_choice, input_type):
   # Supply elasticity
    Es1 = np.array([[0.5, 0], [0, 0.3]])
    Es2 = np.array([[0.3, 0], [0, 0.2]])

    # Demand Elasticity
    # Select the appropriate set
    if a_matrix_choice == 'Independent':
        Ed1 = np.array([[0.5, 0], [0, 0.5]])
        Ed2 = np.array([[0.3, 0], [0, 0.3]])
    elif a_matrix_choice == 'Substitutes':
        Ed1 = np.array([[0.5, 0.3], [0.3, 0.5]])
        Ed2 = np.array([[0.3, 0.2], [0.2, 0.3]])
    elif a_matrix_choice == 'Perfect Substitutes':
        Ed1 = np.array([[0.5, 0.5], [0.5, 0.5]])
        Ed2 = np.array([[0.3, 0.3], [0.3, 0.3]])
    elif a_matrix_choice == 'Complements':
        Ed1 = np.array([[0.5, -0.5], [-0.5, 0.5]])
        Ed2 = np.array([[0.3, -0.3], [-0.3, 0.3]])

    A = np.array([
        [0, 0, -(Es1[0, 0] + Es2[0, 0] + Ed1[0, 0] + Ed2[0, 0]), -(Ed1[0, 1] + Ed2[0, 1])],
        [0, 0, -(Ed1[1, 0] + Ed2[1, 0]), -(Es1[1, 1] + Es2[1, 1] + Ed1[1, 1] + Ed2[1, 1])],
        [k, 0, -R * (Es1[0, 0] + Es2[0, 0] + Ed1[0, 0] + Ed2[0, 0]), -R * (Ed1[0, 1] + Ed2[0, 1])],
        [0, k, -R * (Ed1[1, 0] + Ed2[1, 0]), -R * (Es1[1, 1] + Es2[1, 1] + Ed1[1, 1] + Ed2[1, 1])]
    ])
    B = np.array([[1, 1, 0 ,0],[0,0,1,1],[R,R,0,0], [0,0,R,R]])
    B = np.hstack((B, np.identity(4)))      # stack irrationality noise input matrix

    
    C = np.identity(4)

    t = np.arange(0, 100, .001)

    # Pulse input parameters
    pulse_start = 2  # Start time of the pulse
    pulse_duration = 3  # Duration of the pulse
    # Oscillating input parameters
    amplitude = 5
    frequency = .05
    offset    = 20
    
    # select the input type
    if input_type == 'Step':
        u = 30*np.ones((len(t), 1))  # Step input
    elif input_type == 'Shock':
        u = 20*np.ones((len(t), 1))
        u[(t >= pulse_start) & (t < pulse_start + pulse_duration)] = 30
    elif input_type == 'Ramp':
        u = np.linspace(10, 50, len(t)).reshape(len(t), 1)
    elif input_type == 'Oscillating':
        u = amplitude * np.sin(2 * np.pi * frequency * t).reshape(len(t), 1)+offset
   

   
    
    # Create a matrix of ones with the same number of rows as 'u' and 3 columns
    u2 =  20*np.ones((len(t), 1))
    u3 = u2
    u4 = u3
    
    # Concatenate the original 'u' array with the additional columns of ones
    u_ext = np.hstack((u, u2,u3,u4))
    

    w = np.vstack([
            np.zeros(len(u)),  # First and second rows of zeros
            np.zeros(len(u)),
            np.random.normal(0,sig, len(t)),  # Third row of scaled random noise
            np.zeros(len(u))  # Fourth row of zeros
            ]).T  # Transpose w here
    u_ext = np.hstack([u_ext, w])  # Now, both u_ext and w have the same number of rows
    u_ss = np.array([20, 20, 20, 20,0,0,0,0])  # Constant inputs of 10
    x_ss = np.linalg.solve(-A, B @ u_ss)
        
    dt = t[1] - t[0]

    # Discretize the system (update A_d and B_d based on new 'R')
    I = np.identity(A.shape[0])
    A_d = I + A * dt
    B_d = B * dt
   
   
    # Initialize and run the simulation 
     
    x = np.zeros((A.shape[0], len(t)))

    # Replace the first column of the array x with x_ss
    x[:, 0] = x_ss

    
    for i in range(1, len(t)):
        x[:, i] = A_d @ x[:, i-1] + B_d @ u_ext[i-1,:]
    y = C @ x 
    
    return t, y, u



def update_plot(R, k, sig, a_matrix_choice,input_type):
    t, y, u = system_model(R, k, sig, a_matrix_choice,input_type)
    plt.figure(figsize=(10, 6))
    plt.plot(t, u,  label='Input: Inelastic Demand')
    plt.plot(t, y[0, :], label='Backlog 1')
    plt.plot(t, y[1, :], label='Backlog 2')
    plt.plot(t, y[2, :], label='Price 1')
    plt.plot(t, y[3, :], label='Price 2')
    plt.xlabel('Time')
    plt.ylabel('Output')
    plt.title(f'Market dynamics for R = {R} and k = {k}')
    plt.legend()
    plt.grid(True)
    plt.show()

# Slider for R
R_slider = widgets.FloatSlider(
    value=0.03,  # Initial value
    min=0, max=1,  # Min and max values
    step=0.001,  # Step size
    description='Market friction R:',
    continuous_update=False  # Update only when the slider is released
)

# Slider for k
k_slider = widgets.FloatSlider(
    value=0.1,  # Initial value
    min=0.0000000001, max=1,  # Min and max values
    step=0.01,  # Step size
    description='Storage preference k:',
    continuous_update=False
)

# Slider for sigma
sigma_slider = widgets.FloatSlider(
    value=0,  # Initial value
    min=0.0, max=100,  # Min and max values
    step=0.01,  # Step size
    description='Irrationality:',
    continuous_update=False
)

# Radio buttons for selecting A matrix
a_matrix_selector = widgets.RadioButtons(
    options=['Substitutes', 'Perfect Substitutes', 'Complements','Independent'], 
    description='Types of commodities:', 
    disabled=False
)

# Radio buttons for selecting input type
input_type_selector = widgets.RadioButtons(
    options=['Step', 'Shock', 'Ramp', 'Oscillating'],
    description='Input Type:',
    disabled=False
)



widgets.interactive(update_plot, sig=sigma_slider, R=R_slider, k=k_slider, a_matrix_choice=a_matrix_selector, input_type=input_type_selector)


interactive(children=(FloatSlider(value=0.03, continuous_update=False, description='Market friction R:', max=1…